In [15]:
import pandas as pd 
import numpy as np

def data_comp(variables):

    '''
    cleaning OECD data pass in the name of csv files you are using

    '''
    for n, var in enumerate(variables):

        #read in file OECD data

        df =  pd.read_csv(f"{var}.csv")

        #drop redundent columns and transform it into Year, Country, Variable

        df.columns = df.iloc[2]
        df.drop([0,1,2], inplace=True)
        df.drop(df.columns[1:44], axis=1, inplace=True)
        df.set_index('Country Name', inplace = True, drop = True)
        df = df.T
        df.index = [round(idx) for idx in df.index]
        df.index = pd.to_datetime(df.index, format='%Y')
        df = df.reset_index()
        df = pd.melt(df, id_vars=['index'], value_vars=df.columns[1:], var_name='Country Name', value_name='Value')

        df.set_index(['index', 'Country Name'], inplace=True)
        df.index.names = ["Year", "Country Name"]
        df.rename(columns={'Value': var}, inplace=True)

        #making the dataframe and combining over all variables
        if n == 0: 
            full_df = df
        else: 
            full_df = pd.concat([full_df, df], axis =1)
    return(full_df)

df_ML = data_comp(['GDP', 'Govt_debt_gdp', 'GDP growth rate','GNI per capita',
            'Inflation', 'Lending rate', 'Net exports', 'Total_reserve', 'Forex rate'])

In [16]:
df_PPP_model= data_comp(['RER', 'Forex rate', 'PPP'])
df_PPP = pd.read_csv(f"PPP_US.csv")

df_PPP_model = df_PPP_model[df_PPP_model.notna().all(axis=1)]

df_PPP_model['con_RER'] = (df_PPP_model['Forex rate'] * df_PPP_model['PPP'])

#df_PPP_model.to_csv('.csv', index=True)
df_PPP_model


,,RER,Forex rate,PPP,con_RER
Year,Country Name,,,,
2000-01-01,Armenia,74.010935,539.525833,0.234525,126.532512
2001-01-01,Armenia,73.034791,555.078258,0.231919,128.733275
2002-01-01,Armenia,73.665189,573.353333,0.226287,129.742387
2003-01-01,Armenia,67.937520,578.762954,0.229938,133.079710
2004-01-01,Armenia,71.634581,533.450833,0.258205,137.739693
...,...,...,...,...,...
2017-01-01,Zambia,94.221086,9.517500,0.440513,4.192580
2018-01-01,Zambia,88.528850,10.458333,0.420550,4.398256
2019-01-01,Zambia,79.795053,12.890000,0.360806,4.650791


In [11]:

df= pd.read_csv(f"PPP_US.csv")

country_mapping = {

    'ARM': 'Armenia', 'ATG': 'Antigua and Barbuda', 'AUS': 'Australia', 'BDI': 'Burundi', 'BGR': 'Bulgaria', 
    'BHR': 'Bahrain', 'BHS': 'Bahamas, The', 'BLZ': 'Belize', 'BOL': 'Bolivia', 'BRA': 'Brazil', 
    'CAF': 'Central African Republic', 'CAN': 'Canada', 'CHE': 'Switzerland', 'CHL': 'Chile', 'CHN': 'China', 
    'CIV': "Cote d'Ivoire", 'CMR': 'Cameroon', 'COD': 'Congo, Dem. Rep.', 'COL': 'Colombia', 'CRI': 'Costa Rica', 
    'CYP': 'Cyprus', 'CZE': 'Czechia', 'DMA': 'Dominica', 'DNK': 'Denmark', 'DOM': 'Dominican Republic', 
    'DZA': 'Algeria', 'FJI': 'Fiji', 'GAB': 'Gabon', 'GBR': 'United Kingdom', 'GEO': 'Georgia', 'GHA': 'Ghana', 
    'GMB': 'Gambia, The', 'GNQ': 'Equatorial Guinea', 'GRC': 'Greece', 'GRD': 'Grenada', 'GUY': 'Guyana', 
    'HKG': 'Hong Kong SAR, China', 'HRV': 'Croatia', 'HUN': 'Hungary', 'IRN': 'Iran, Islamic Rep.', 'ISL': 'Iceland', 
    'ISR': 'Israel', 'JPN': 'Japan', 'KNA': 'St. Kitts and Nevis', 'KOR': 'Korea, Rep.', 'LCA': 'St. Lucia', 
    'LSO': 'Lesotho', 'LVA': 'Latvia', 'MAR': 'Morocco', 'MDA': 'Moldova', 'MEX': 'Mexico', 'MKD': 'North Macedonia', 
    'MLT': 'Malta', 'MWI': 'Malawi', 'MYS': 'Malaysia', 'NGA': 'Nigeria', 'NIC': 'Nicaragua', 'NOR': 'Norway', 
    'NZL': 'New Zealand', 'PAK': 'Pakistan', 'PHL': 'Philippines', 'PNG': 'Papua New Guinea', 'POL': 'Poland', 
    'PRY': 'Paraguay', 'ROU': 'Romania', 'RUS': 'Russian Federation', 'SAU': 'Saudi Arabia', 'SGP': 'Singapore', 
    'SLB': 'Solomon Islands', 'SLE': 'Sierra Leone', 'SVK': 'Slovak Republic', 'SWE': 'Sweden', 'TGO': 'Togo', 
    'TTO': 'Trinidad and Tobago', 'TUN': 'Tunisia', 'UGA': 'Uganda', 'UKR': 'Ukraine', 'URY': 'Uruguay', 'USA': 'United States',
    'VCT': 'St. Vincent and the Grenadines', 'VEN': 'Venezuela, RB', 'WSM': 'Samoa', 'ZAF': 'South Africa',
    'ZMB': 'Zambia'}

def apply_country_mapping(location):
    if location in country_mapping:
        return country_mapping[location]
    else:
        return None

df['LOCATION'] = df['LOCATION'].apply(apply_country_mapping)

# Extract only the necessary columns
df = df[['LOCATION', 'TIME', 'Value']]
df['TIME'] = pd.to_datetime(df['TIME'], format='%Y')

df = df.rename(columns={'Value': 'PPP_for_ER'})
df = df.dropna()
df.set_index(['TIME', 'LOCATION'], inplace=True)
df.index.set_names(['Year', 'Country Name'], inplace=True)

df

,,PPP_for_ER
Year,Country Name,
1960-01-01,Australia,0.700946
1961-01-01,Australia,0.701324
1962-01-01,Australia,0.687990
1963-01-01,Australia,0.698321
1964-01-01,Australia,0.708346
...,...,...
2017-01-01,Cameroon,232.801392
2018-01-01,Cameroon,231.682690
2019-01-01,Cameroon,230.273904


In [14]:
df
df_merged = df.merge(df_PPP_model, left_index=True, right_index=True, how='inner')

#theretically in the long run these would be zero since the NOM*PPP == RER_normed
#so now we will calculate the difference between the constructed Real exchange rate where RER = PPP*NER
#versus the calculated PPP

#if the PPP*nom is higher than the PPP based on the exchange rate the currency would be overvalued
#id PPP *nom is lower than PPP the currency would be undervalued

#a lot of issues because I dont know the methodology of how the real excahnge rate of PPP was calculated for the country which I grabbed
#from OCED but still shows what we want to buy or sell depending on the year

df_merged['Final_diff'] = df_merged['con_RER'] - df_merged['PPP_for_ER']

#filtering out unstable ones
df_merged = df_merged[df_merged['Final_diff']>-0.1]
df_merged = df_merged[df_merged['Final_diff']<0.1]

print(df_merged)
df_merged.sort_index(inplace=True)

# Select all rows with years before 2015 and including 2015
df_year_before_2015 = df_merged.loc[(slice(None, '2015-01-01'), slice(None)), :]




df_merged['diff_in_ave_RER'] = df_merged['RER'].div(df_year_before_2015.groupby('Country Name').mean()['RER'])

df_years={}
for yr in range(2000, 2022):
    name = f'df_{yr}'
    dt_year= pd.to_datetime(f'{yr}-01-01')
    df_year = df_merged.loc[(dt_year, slice(None))]
    df_years[yr] = df_year
    #df_years[yr].to_csv(f'df_years_{yr}.csv', index=True)
print(df_years[2016])


                         PPP_for_ER        RER  Forex rate       PPP   
Year       Country Name                                                
2001-01-01 Australia       1.327544  67.566904    1.933442  0.712432  \
2002-01-01 Australia       1.336490  70.857304    1.840563  0.699294   
2004-01-01 Australia       1.364771  85.422902    1.359752  0.970814   
2005-01-01 Australia       1.388356  87.966700    1.309473  1.043720   
2006-01-01 Australia       1.402633  87.241236    1.327973  1.050189   
...                             ...        ...         ...       ...   
2017-01-01 Cameroon      232.801392  96.194763  580.656750  0.400928   
2018-01-01 Cameroon      231.682690  97.626845  555.446458  0.417111   
2019-01-01 Cameroon      230.273904  96.575057  585.911013  0.393019   
2020-01-01 Cameroon      229.137013  99.436060  575.586004  0.398093   
2021-01-01 Cameroon      227.378726  99.551963  554.530675  0.410038   

                            con_RER  Final_diff  
Year       Co